In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import scipy
import scipy.stats as stats
import os

In [2]:
# project files

from utils.variables import pack, unpack
from utils.display import table, tabulate
from utils.files import download_sheet, save, load
from utils.hash import digest

from fit_options import fit_options
from game_model import game
from rp_model import compute_rp, make_precomputed_columns
from initial_guess import make_initial_guess


In [3]:
# stuff for display

from IPython.lib.pretty import pretty, pprint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [4]:
# Load previous data

data = pd.read_pickle(fit_options.data_file)
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,FreqL,Inv,Berry1,BerryL,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,BerryD,IngD,Dupes,Amnt,Ing2P,Help skill bonus,RP Multiplier
count,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.000000,6251.00000,6251.000000,6251.000000,6251.000000,6251.000000,6185.000000,6251.000000,6251.000000,6251.000000,6251.0,6251.000000,6251.000000,6251.000000,6251.000000
mean,10.539754,604.216765,1.490162,598.748071,0.018907,3957.958727,3808.092725,12.225724,28.039674,37.679251,108.799072,1.01804,0.988365,0.994420,1.001760,1.005919,0.207669,32.246559,55.161894,148.545993,1.0,0.073268,2.998080,0.987581,1.027236
std,7.143925,389.951541,0.983231,395.586862,0.987593,962.682216,1024.731656,3.384683,2.612284,8.012329,15.999728,0.30429,0.059484,0.044932,0.117807,0.117834,0.046656,36.406872,27.743413,60.567634,0.0,0.514501,20.567088,0.037777,0.069380
min,1.000000,245.000000,1.000000,-1.000000,-12.780000,2200.000000,1489.118400,7.000000,24.000000,24.000000,90.000000,0.52000,0.890000,0.920000,0.800000,0.800000,0.000000,0.000000,25.000000,90.000000,1.0,0.000000,0.000000,0.790000,1.000000
25%,5.000000,354.000000,1.000000,352.475000,-0.290000,3300.000000,2954.070000,10.000000,26.000000,32.000000,98.000000,0.80000,1.000000,1.000000,1.000000,1.000000,0.180000,12.820000,35.000000,101.000000,1.0,0.000000,0.000000,1.000000,1.000000
50%,10.000000,460.000000,1.000000,458.090000,-0.020000,3900.000000,3796.100000,12.000000,28.000000,36.000000,103.000000,0.94000,1.000000,1.000000,1.000000,1.000000,0.200000,19.750000,45.000000,121.000000,1.0,0.000000,0.000000,1.000000,1.000000
75%,14.000000,711.000000,2.000000,711.185000,0.270000,4500.000000,4455.000000,14.000000,31.000000,41.000000,115.000000,1.21000,1.000000,1.000000,1.000000,1.000000,0.240000,43.980000,68.000000,202.000000,1.0,0.000000,0.000000,1.000000,1.000000
max,42.000000,3440.000000,6.000000,3440.330000,15.410000,6300.000000,6916.140000,23.000000,35.000000,77.000000,151.000000,2.41000,1.120000,1.080000,1.200000,1.200000,0.420000,414.030000,222.000000,511.000000,1.0,8.000000,342.000000,1.000000,1.440000


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
0,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.41,0.41,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,3700.0,3276.72,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,1.09,0.90,1.00,1.0,0.8,0.26,22.76,538.37,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
1,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,530.69,-0.31,Neutral,ArbokNeutral81,-,-,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.0,1.0,0.26,22.76,531.00,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
2,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.08,0.08,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.2,1.0,0.26,22.76,530.61,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
3,Arcanine,5.0,958.0,Calm,3.0,,957.85,-0.15,Calm,ArcanineCalm53,Main Skill Chance,Speed of Help,2500.0,2728.00,Fire,Skills,Extra Helpful S,16.0,27.0,31,Fiery Herb,130.0,1.31,1.11,1.00,1.0,1.2,0.14,84.52,937.81,ArcanineNeutral53,31.0,130.0,1,True,,,,0.0,0.0,1.0,1.0
4,Arcanine,3.0,715.0,Lax,2.0,pokemon sleep general,714.65,-0.35,Lax,ArcanineLax32,Energy Recovery,Main Skill Chance,2500.0,2490.00,Fire,Skills,Extra Helpful S,16.0,27.0,29,Fiery Herb,130.0,1.44,1.00,1.08,1.0,0.8,0.14,61.23,750.25,ArcanineNeutral32,29.0,130.0,1,True,,,,0.0,0.0,1.0,1.0


In [5]:
# Load previous fit

x0, unpack_info = pack(make_initial_guess())

hash_value = digest(data, x0)
filename = fit_options.result_file(hash_value)

opt = load(filename)
sol = unpack(opt.x, unpack_info)

opt

    message: `ftol` termination condition is satisfied.
    success: True
     status: 2
          x: [ 2.563e-01  2.545e-01 ...  2.195e-01  9.953e-01]
       cost: 1119.3668854526234
       grad: [ 2.956e-05 -3.122e-05 ...  8.217e-05 -7.706e-05]
 optimality: 2.1351560882176273
       nfev: 21
       njev: 16

In [6]:
# Helpers

def truncated_normal_sample(size, mu, sigma, lower, upper):
    return stats.truncnorm.rvs( (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma, size=size)

def round_noise(size):
    eps = 1e-6
    return truncated_normal_sample(size, mu=0.0, sigma=0.25, lower= -0.5+eps, upper= 0.5-eps)

def harmonic(a,b):
    return 2.0 / ( 1.0 / a + 1.0 / b )

In [7]:
# In the boostrap method,
# We create N synthetic datasets that are likely to represent the population
# Then we redo the fit on those, starting from the best fit in initial dataset.
# Finally, we collect statistics on the fitted variables between runs

# To create the synthetic datasets, we use sampling with replacement (allow duplicate)
# Because some Pokémon are rare, we use stratified sampling

stratas = data.groupby(['Pokemon'], group_keys=False)
target_group_size =  stratas.size().quantile(q=0.75)
    

n_boostraps = 50
all_opt_x = [opt.x]

In [8]:
cache_fit = True
boostrap_filename = f"./results/bootstrap-fit-{hash_value}.pickle"

if cache_fit and os.path.isfile(boostrap_filename):

    all_opt_x = load(boostrap_filename)
    print("Loaded from cache")

else:

    # WARNING: RUNNING THE OPTIMISATION N TIMES IS VERY LONG.
    #
    # YOU CAN INTERRUPT THE KERNEL (STOP BUTTON)
    # AND RUN THE NEXT CELLS IF YOU WANT TO SEE CURRENT PROGRESS
    #
    # THE COLLECTION `all_opt_x` IS DEFINED ABOVE THIS CELL
    # SO YOU CAN RESUME ADDING MORE RUNS

    while( len(all_opt_x) < n_boostraps ):

        print( "Boostrap run " + str(len(all_opt_x)) )

        # To create the synthetic datasets, we use sampling with replacement (allow duplicate)

        # We'll introduce a correction toward more equal sample size between Pokémon
        # We can motivate that by the fact stratified sampling should use population proportions
        # instead of current data proportions

        resampled = stratas.apply(lambda x: x.sample( round( harmonic(len(x),target_group_size) ), replace=True, ignore_index=True))

        # Add small noise to RP
        # We do so to simulate some unknown value that would round() to current RP

        referenceRP = resampled["RP"].to_numpy()
        referenceRP += round_noise(len(resampled))

        # Compute per sample information about help time, nature, sub-skills etc
        recomputed = make_precomputed_columns(resampled)

        # Put the pieces together

        def residual(x):
            return referenceRP - compute_rp(x, resampled, recomputed, unpack_info)

        # Redo the fit, starting from the optimal we found on current data
        opt2 = scipy.optimize.least_squares(residual, opt.x, **fit_options.least_squares_kwargs)

        # Collect results for stats
        all_opt_x.append(opt2.x)

    # Save
    save(boostrap_filename, all_opt_x)

Boostrap run 1
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.3804e+03                                    3.50e+06    
       1              2         1.3266e+03      5.38e+01       9.59e-01       1.63e+06    
       2              7         1.3228e+03      3.84e+00       1.75e-03       5.91e+05    
       3              8         1.3206e+03      2.18e+00       1.75e-03       1.41e+05    
       4              9         1.3203e+03      3.14e-01       3.50e-03       1.96e+03    
       5             10         1.3200e+03      2.70e-01       7.00e-03       6.13e+03    
       6             11         1.3197e+03      3.66e-01       1.40e-02       5.05e+03    
       7             12         1.3191e+03      5.80e-01       2.80e-02       1.58e+03    
       8             13         1.3182e+03      9.14e-01       5.60e-02       1.71e+03    
       9             14         1.3173e+03      8.64e-01       1.12e-01    

In [9]:
# Stats
# We'll use the median as estimate for the mean.
# And 1.4826 * < median absolute deviation from the median > as estimate for std

center = np.median(all_opt_x, axis=0)

interval_std = 1.96 * np.std( all_opt_x, axis=0) # 95% confidence region
interval_mad = 3 * np.median( list(map(lambda x: np.abs(center-x), all_opt_x)), axis=0) # 95% confidence region

sol_center = unpack(center, unpack_info)
sol_interval_std = unpack(interval_std, unpack_info)
sol_interval_mad = unpack(interval_mad, unpack_info)

# Pretty display
pd.DataFrame({
    
    "Pokemon":game.pokedex.data["Pokemon"], 
    
    "ing%": sol_center["Pokemons ing fractions"],
    #"conf (ing)": sol_interval_std["Pokemons ing fractions"]*100.0,
    "conf (ing)*": sol_interval_mad["Pokemons ing fractions"],

    "skill% * skillValue": sol_center["Pokemons skill products"], 
    #"conf (skill)": sol_interval_std["Pokemons skill products"],
    "conf (skill)*": sol_interval_mad["Pokemons skill products"],


}).set_index("Pokemon")

,ing%,conf (ing)*,skill% * skillValue,conf (skill)*
Pokemon,,,,
Bulbasaur,0.256320,0.000542,16.711230,0.069210
Ivysaur,0.254479,0.000389,16.696585,0.030532
Venusaur,0.265769,0.000556,18.416059,0.080335
Charmander,0.200457,0.000353,9.669925,0.044419
Charmeleon,0.226517,0.000447,14.054431,0.039153
Charizard,0.223958,0.000381,14.014227,0.051138
Squirtle,0.270087,0.000456,17.614872,0.054855
Wartortle,0.270457,0.000238,17.578021,0.027369
Blastoise,0.274338,0.000377,18.431467,0.027343


In [10]:
# Explain what we did with the re-sampling target size
# pd.DataFrame({'before': stratas.size(), 'after': stratas.apply(lambda x: round( harmonic(len(x), target_group_size) )) })

In [11]:
data[data["Pokemon"]=="Slaking"]

,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
2274,Slaking,27.0,2648.0,Calm,6.0,,2645.62,-2.38,Calm,SlakingCalm276Skill Level Up MEnergy Recovery ...,Main Skill Chance,Speed of Help,3800.0,3962.640,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.90,1.10,1.00,1.0,1.2,0.34,304.43,2545.01,SlakingNeutral276Skill Level Up MEnergy Recove...,108.0,110.0,1,True,Skill Level Up M,Energy Recovery Bonus,,0.0,0.0,1.00,1.22
2275,Slaking,27.0,1615.0,Gentle,3.0,,1616.87,1.87,Gentle,SlakingGentle273Berry Finding SHelping bonus,Main Skill Chance,Energy Recovery,3800.0,3602.400,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.99,1.00,0.92,1.0,1.2,0.34,115.61,1616.08,SlakingNeutral273Berry Finding SHelping bonus,162.0,110.0,1,True,Berry Finding S,Helping bonus,,0.0,0.0,1.00,1.22
2276,Slaking,27.0,1537.0,Naughty,3.0,https://discord.com/channels/11339232641467064...,1537.50,0.50,Naughty,SlakingNaughty273Berry Finding SInventory Up M,Speed of Help,Main Skill Chance,3800.0,3242.160,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,1.11,0.89,1.00,1.0,0.8,0.34,115.61,1500.20,SlakingNeutral273Berry Finding SInventory Up M,162.0,110.0,1,True,Berry Finding S,Inventory Up M,,0.0,0.0,1.00,1.13
2277,Slaking,22.0,965.0,Serious,1.0,https://discord.com/channels/11339232641467064...,964.57,-0.43,Neutral,SlakingNeutral221Inventory Up L,-,-,3800.0,3640.400,Normal,Berries,Ingredient Magnet S,12.0,28.0,49,Snoozy Tomato,110.0,0.98,1.00,1.00,1.0,1.0,0.34,58.90,965.00,SlakingNeutral221Inventory Up L,98.0,110.0,1,True,Inventory Up L,,,0.0,0.0,1.00,1.18
2278,Slaking,25.0,1009.0,Hardy,1.0,https://cdn.discordapp.com/attachments/1156242...,1009.73,0.73,Neutral,SlakingNeutral251Skill Trigger SSkill Trigger M,-,-,3800.0,3617.600,Normal,Berries,Ingredient Magnet S,12.0,28.0,52,Snoozy Tomato,110.0,0.99,1.00,1.00,1.0,1.0,0.34,90.71,1009.00,SlakingNeutral251Skill Trigger SSkill Trigger M,104.0,110.0,1,True,Skill Trigger S,Skill Trigger M,,0.0,0.0,1.00,1.00
2279,Slaking,27.0,2637.0,Quiet,6.0,,2636.77,-0.23,Quiet,SlakingQuiet276Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3602.400,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.99,1.00,1.00,1.2,1.0,0.34,414.03,2628.21,SlakingNeutral276Skill Level Up MSkill Trigger M,108.0,110.0,1,True,Skill Level Up M,Skill Trigger M,,0.0,0.0,1.00,1.00
2280,Slaking,29.0,2681.0,Quiet,6.0,,2681.36,0.36,Quiet,SlakingQuiet296Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3587.200,Normal,Berries,Ingredient Magnet S,12.0,28.0,56,Snoozy Tomato,110.0,1.00,1.00,1.00,1.2,1.0,0.34,414.03,2672.46,SlakingNeutral296Skill Level Up MSkill Trigger M,112.0,110.0,1,True,Skill Level Up M,Skill Trigger M,,0.0,0.0,1.00,1.00
2281,Slaking,30.0,2831.0,Quiet,6.0,,2831.79,0.79,Quiet,SlakingQuiet306Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3579.600,Normal,Berries,Ingredient Magnet S,12.0,28.0,57,Snoozy Tomato,110.0,1.00,1.00,1.00,1.2,1.0,0.34,414.03,2799.06,SlakingNeutral306Skill Level Up MSkill Trigger M,114.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.00,1.00
2282,Slaking,32.0,2859.0,Quiet,6.0,,2859.48,0.48,Quiet,SlakingQuiet326Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3564.400,Normal,Berries,Ingredient Magnet S,12.0,28.0,60,Snoozy Tomato,110.0,1.00,1.00,1.00,1.2,1.0,0.34,414.03,2828.99,SlakingNeutral326Skill Level Up MSkill Trigger M,120.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.00,1.00
2283,Slaking,33.0,2905.0,Quiet,6.0,,2905.24,0.24,Quiet,SlakingQuiet336Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3556.800,Normal,Berries,I

In [12]:
data[data["Pokemon"]=="Glaceon"]


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
1288,Glaceon,9.0,773.0,Lonely,2.0,pokemon sleep general,773.23,0.23,Lonely,GlaceonLonely92,Speed of Help,Energy Recovery,3200.0,2833.92,Ice,Skills,Cooking Power-Up S,12.0,32.0,40,Moomoo Milk,98.0,1.27,0.9,0.92,1.0,1.0,0.22,78.63,754.21,GlaceonNeutral92,40.0,98.0,1,True,,,,0.0,0.0,1.0,1.00
1289,Glaceon,9.0,815.0,Relaxed,2.0,,814.79,-0.21,Relaxed,GlaceonRelaxed92,Energy Recovery,EXP Gains,3200.0,3148.80,Ice,Skills,Cooking Power-Up S,12.0,32.0,40,Moomoo Milk,98.0,1.14,1.0,1.08,1.0,1.0,0.22,78.63,754.63,GlaceonNeutral92,40.0,98.0,1,True,,,,0.0,0.0,1.0,1.00
1287,Glaceon,10.0,1230.0,Gentle,4.0,https://discord.com/channels/11339232641467064...,1230.04,0.04,Gentle,GlaceonGentle104Skill Level Up M,Main Skill Chance,Energy Recovery,3200.0,3142.40,Ice,Skills,Cooking Power-Up S,12.0,32.0,41,Moomoo Milk,98.0,1.14,1.0,0.92,1.0,1.2,0.22,149.93,1166.04,GlaceonNeutral104Skill Level Up M,41.0,98.0,1,True,Skill Level Up M,,,0.0,0.0,1.0,1.00
1288,Glaceon,11.0,1235.0,Gentle,4.0,https://discord.com/channels/11339232641467064...,1234.93,-0.07,Gentle,GlaceonGentle114Skill Level Up M,Main Skill Chance,Energy Recovery,3200.0,3136.00,Ice,Skills,Cooking Power-Up S,12.0,32.0,42,Moomoo Milk,98.0,1.14,1.0,0.92,1.0,1.2,0.22,149.93,1171.46,GlaceonNeutral114Skill Level Up M,42.0,98.0,1,True,Skill Level Up M,,,0.0,0.0,1.0,1.00
1289,Glaceon,12.0,1251.0,Gentle,4.0,https://discord.com/channels/11339232641467064...,1250.73,-0.27,Gentle,GlaceonGentle124Skill Level Up M,Main Skill Chance,Energy Recovery,3200.0,3129.60,Ice,Skills,Cooking Power-Up S,12.0,32.0,43,Moomoo Milk,98.0,1.15,1.0,0.92,1.0,1.2,0.22,149.93,1187.33,GlaceonNeutral124Skill Level Up M,43.0,98.0,1,True,Skill Level Up M,,,0.0,0.0,1.0,1.00
1290,Glaceon,14.0,891.0,Bashful,2.0,,891.00,0.00,Neutral,GlaceonNeutral142Inventory Up M,-,-,3200.0,3116.80,Ice,Skills,Cooking Power-Up S,12.0,32.0,45,Moomoo Milk,98.0,1.15,1.0,1.00,1.0,1.0,0.22,78.63,891.00,GlaceonNeutral142Inventory Up M,45.0,98.0,1,True,Inventory Up M,,,0.0,0.0,1.0,1.13
